In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as dt

In [2]:
#method to generate random samples following Normal distribution in d dimensions
def multi_gaussian(dim, num_of_samples):
    cov_matrix = dt.make_spd_matrix(dim) #generate random symmetric, positive definite matrix
    mean = np.random.rand(dim)
    x = np.random.multivariate_normal(mean, cov_matrix, (num_of_samples)) #shape of x is num_of_samples x dim
    return x
# multi_gaussian(5,10)

In [3]:
def discriminant_function(x, mean_vec, cov_matrix, dim, prior_prob):
        if dim > 1: # multivariate gaussian
            cov_inverse = np.linalg.inv(cov_matrix)
            cov_det = np.linalg.det(cov_matrix)
            return (
                    ((- 1/2) * (mahalanobis_dist(x, mean_vec, cov_inverse))**2) \
                    - ((dim / 2) * ( np.log(2 * np.pi)) )\
                    - ( (1/2) * np.log(cov_det) ) \
                    + np.log(prior_prob)
            )
        else: # univariate gaussian
#             print("euc_dist",euclidean_dis(x, mean_vec))
#             print(cov_matrix)
            return (
                    -(1/2) * euclidean_dis(x, mean_vec)**2 /  cov_matrix \
                    - (1/2) * (np.log(2 * np.pi * cov_matrix))\
                    + np.log(prior_prob)
            )

In [4]:
#calculate Euclidean distance between 2 points x and y
def euclidean_dis(x, mean_vector):
    cov_inverse = np.identity(x.shape[1])
    return mahalanobis_dist(x, mean_vector, cov_inverse) #

In [5]:
def mahalanobis_dist(x, mean_vector, cov_inverse):
#     print("shape of x in mahalanobis dist", x.shape)
    diff = ( x - mean_vector )
    return ( np.dot( np.dot(diff, cov_inverse), diff.T ) ) ** (1/2)

# Section

In [6]:
data = np.genfromtxt('data_dhs_chap2.csv', delimiter=',', skip_header=1)
data.shape

(30, 4)

In [7]:
def dichotomizer(X, Y, dim):
    if dim == 1 : # only 1 independent feature
        class1_mean_vec = np.mean( X[:5] )
        class2_mean_vec = np.mean( X[5:10] )
        class1_cov = np.cov( X[:5] )
        class2_cov = np.cov( X[5:10] )
        shape = (1,1)
    else: # more than 1
        class1_mean_vec = np.mean( X[:5], axis=0 )
        class2_mean_vec = np.mean( X[5:10], axis=0 )
        class1_cov = np.cov( X[:5].T )
        class2_cov = np.cov( X[5:10].T )
        shape = (dim,)
    class1_prior_prob = 0.5
    class2_prior_prob = 0.5
    predicted_class = []
    for instance in X:
        instance = instance.reshape( shape )
        g1 = discriminant_function(instance, class1_mean_vec, class1_cov, dim, class1_prior_prob)
        g2 = discriminant_function(instance, class2_mean_vec, class2_cov, dim, class2_prior_prob)
        if g1 > g2 :
            predicted_class.append(0) # class 1
        else :
            predicted_class.append(1) # class 2
    return predicted_class

In [8]:
def empirical_training_error(target_class, predicted_class):
    total_instances = len(predicted_class)
    error = 0 
    for instance in range(total_instances) :
        error += np.abs( target_class[instance] - predicted_class[instance] )
    avg_error_percent = (100 / total_instances) * error
    return avg_error_percent

In [9]:
trg_data =  data[ 5:15 , [0,3] ] 
X = trg_data[:, 0] 
Y = trg_data[:, 1].astype(int)
target_class = []
for i in Y:
    target_class.append(i.item())
predicted_class = dichotomizer(X, Y, 1)
print("predicted class labels:", predicted_class)
print("misclassification error: " , empirical_training_error(target_class, predicted_class) )

predicted class labels: [0, 1, 1, 0, 0, 0, 0, 1, 1, 0]
misclassification error:  50.0


In [10]:
trg_data =  data[ 5:15 , [0,1,3] ] 
X = trg_data[:, :2] 
Y = trg_data[:, -1].astype(int)
target_class = []
for i in Y:
    target_class.append(i.item())
predicted_class = dichotomizer(X, Y, 2)
print("predicted class labels:", predicted_class)
print("misclassification error: " , empirical_training_error(target_class, predicted_class) )

predicted class labels: [0, 1, 1, 0, 0, 0, 0, 1, 1, 1]
misclassification error:  40.0


In [11]:
trg_data =  data[ 5:15 , [0,1,2,3] ] 
X = trg_data[:, :3] 
Y = trg_data[:, 3].astype(int)
target_class = []
for i in Y:
    target_class.append(i.item())
predicted_class = dichotomizer(X, Y, 3)
print("predicted class labels:", predicted_class)
print("misclassification error: " , empirical_training_error(target_class, predicted_class) )

predicted class labels: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
misclassification error:  0.0


In [12]:
test_data = np.array([
    [1,2,1],
    [5,3,2],
    [0,0,0],
    [1,0,0],
])

class1_samples = data[:10,[0,1,2]]
class2_samples = data[:20,[0,1,2]]
class3_samples = data[:30,[0,1,2]]

class1_mean = np.mean(class1_samples, axis=0)
class2_mean = np.mean(class2_samples, axis=0)
class3_mean = np.mean(class3_samples, axis=0)

class1_cov = np.cov(class1_samples.T)
class2_cov = np.cov(class2_samples.T)
class3_cov = np.cov(class3_samples.T)

predicted_class = []
for x in test_data:
    distances = []
    distances.append( mahalanobis_dist(x, class1_mean, class1_cov) )
    distances.append( mahalanobis_dist(x, class2_mean, class2_cov) )
    distances.append( mahalanobis_dist(x, class3_mean, class3_cov) )
#     print(distances)
    predicted_class.append(distances.index(min(distances)))

# print(predicted_class)
print(class1_cov)
print(class2_cov)
print(class3_cov)

[[ 14.38051111   7.69537778   4.12232222]
 [  7.69537778  14.62312111   3.90684   ]
 [  4.12232222   3.90684     19.72453778]]
[[ 24.26008711   8.34623342  -5.80606421]
 [  8.34623342  13.42083658   2.10255053]
 [ -5.80606421   2.10255053  18.08224632]]
[[ 22.87101609  10.42504368   2.52351264]
 [ 10.42504368  13.0415592    6.33164713]
 [  2.52351264   6.33164713  27.66965471]]


# Working with Iris Dataset

Linear Discriminant Function- Case 1- covariance matrices are equal and proportional to Identity matrix

In [13]:
data = np.genfromtxt('iris.csv', delimiter=',')
X = data[:,:4]
label_to_nominal = {
    'Iris-setosa' : 0,
    'Iris-versicolor' : 1,
    'Iris-virginica' : 2
}
class1 = np.full((1,50), label_to_nominal['Iris-setosa'], dtype=int) # 
class2 = np.full((1,50), label_to_nominal['Iris-versicolor'], dtype=int) 
class3 = np.full((1,50), label_to_nominal['Iris-virginica'], dtype=int)
Y = np.append(class1, class2)
Y = np.append(Y, class3)

In [19]:
class1_samples = X[0:50,]
class2_samples = X[50:100,]
class3_samples = X[100:150,]

class1_mean = np.mean(class1_samples, axis=0)
class2_mean = np.mean(class2_samples, axis=0)
class3_mean = np.mean(class3_samples, axis=0)

class1_cov = np.cov(class1_samples.T)
class2_cov = np.cov(class2_samples.T)
class3_cov = np.cov(class3_samples.T)

class1_mean.shape

(4,)

In [21]:
def quad_discriminant(x, mean_vec, cov_mat, prior_prob):
    cov_inverse = np.linalg.inv(cov_matrix)
    cov_det = np.linalg.det(cov_matrix)
    
    Wi = (-1 / 2) * (cov_inverse)
    wi = np.dot(cov_inverse, mean_vec)
    wio = (-1 / 2) * ( np.dot(np.dot(mean_vec.T, cov_inverse), mean_vec) + np.log(cov_det) ) + np.log(prior_prob) 
    
    g = np.dot(np.dot(x.T, Wi), x) + np.dot(wi.T, x) + wio
    return g